In [1]:
from __future__ import (print_function, absolute_import)
from __future__ import (division, unicode_literals)

# importing

import numpy as np
from astropy.io import fits
from astropy import units
from glob import glob
import os
import argparse

from pypeit import ginga
from pypeit.core import pixels
from pypeit import traceslits
from pypeit import processimages
from pypeit import scienceimage
from pypeit import arcimage
from pypeit.core import arc
from pypeit import wavecalib
# from pypit import wavecalib
from pypeit import wavetilts
from pypeit import waveimage
from pypeit import flatfield
from pypeit import traceimage
from pypeit import specobjs
from pypeit import utils

# Spectrgraph and Settings
from pypeit.spectrographs.util import load_spectrograph
from pypeit.par import pypeitpar
from pypeit import ginga
from pypeit.core.skysub import global_skysub
from pypeit.core import skysub
from pypeit.core import extract
from pypeit.core.flux_feige import generate_sensfunc
#from pypeit.core.flux import generate_sensfunc
from linetools import utils as ltu

from IPython import embed
from matplotlib import pyplot as plt

In [2]:
spectro_name = 'magellan_fire'
spectrograph = load_spectrograph(spectrograph=spectro_name)

### Load settings

## Detector settings
par = spectrograph.default_pypeit_par()

## Trace settings
par['calibrations']['traceframe']['process']['overscan'] = 'median'

# Arc settings
par['calibrations']['arcframe']['process']['overscan'] = 'median'

# Flat settings
par['calibrations']['pixelflatframe']['process']['overscan'] = 'median'


# Science settings
par['scienceframe']['process']['overscan'] = 'median'
par['calibrations']['slits']

Parameter       Value       Default     Type        Callable
------------------------------------------------------------
function        legendre    legendre    str         False   
polyorder       5           3           int         False   
medrep          0           0           int         False   
number          -1          -1          int         False   
trim            3, 3        3, 3        tuple       False   
maxgap          None        None        int         False   
maxshift        0.5         0.15        int, float  False   
pad             0           0           int         False   
sigdetect       50          20.0        int, float  False   
fracignore      0.01        0.01        float       False   
min_slit_width  6.0         6.0         float       False   
diffpolyorder   2           2           int         False   
single          []          []          list        False   
sobel_mode      nearest     nearest     str         False   
pcatype         pixel   

In [3]:
flat_files = ['/Users/feige/Desktop/FIRE/fire_0038.fits.gz',\
              '/Users/feige/Desktop/FIRE/fire_0039.fits.gz',\
              '/Users/feige/Desktop/FIRE/fire_0040.fits.gz',\
              '/Users/feige/Desktop/FIRE/fire_0041.fits.gz',\
              '/Users/feige/Desktop/FIRE/fire_0042.fits.gz']

sciAfiles = ['/Users/feige/Desktop/FIRE/fire_0029.fits.gz']
sciBfiles = ['/Users/feige/Desktop/FIRE/fire_0030.fits.gz']

In [4]:
tImage = traceimage.TraceImage(spectrograph,
                               file_list=flat_files,
                               par=par['calibrations']['traceframe'])

In [5]:
tflat = tImage.process(bias_subtract='overscan',
                           trim=False, apply_gain=True)
mstrace = tflat.copy()

[INFO]    :: Bias subtracting your image(s)
[INFO]    :: Using overscan to subtact
[INFO]    :: Using overscan to subtact
[INFO]    :: Using overscan to subtact
[INFO]    :: Using overscan to subtact
[INFO]    :: Using overscan to subtact
[INFO]    :: Combining 5 trace_image frames
[INFO]    :: Finding saturated and non-linear pixels
[INFO]    :: Rejecting cosmic rays
[INFO]    :: Not rejecting any low/high pixels
[INFO]    :: Rejecting deviant pixels
[INFO]    :: Combining frames with a weightmean operation
[INFO]    :: Replacing completely masked pixels with the maxnonsat value of the input frames
[INFO]    :: 5 trace_image frames combined successfully!
[WARNING] :: Should probably be measuring the gain across the amplifier boundary


In [6]:
    pixlocn = pixels.gen_pixloc(tImage.stack.shape)
    bpm = spectrograph.bpm(shape=tflat.shape, det=1)

[INFO]    :: Deriving physical pixel locations on the detector
[INFO]    :: Pixel gap in the dispersion direction = 0.000
[INFO]    :: Pixel size in the dispersion direction = 1.000
[INFO]    :: Pixel gap in the spatial direction = 0.000
[INFO]    :: Pixel size in the spatial direction = 1.000
[INFO]    :: Saving pixel locations
[INFO]    :: Custom bad pixel mask for FIRE


In [7]:
ginga.show_image(bpm)
_, _ = ginga.show_image(tflat*(1.-bpm))

In [8]:
par['calibrations']['slits']['polyorder'] = 5
par['calibrations']['slits']['pcatype'] = 'pixel'
par['calibrations']['slits']['pcapar'] = [4,3,2,1,0]
par['calibrations']['slits']['sigdetect'] = 50
par['calibrations']['slits']['maxshift'] = 0.5

#par['calibrations']['slits']

In [9]:
    tSlits = traceslits.TraceSlits(mstrace,
                                   pixlocn,
                                   par=par['calibrations']['slits'],
                                   binbpx=bpm)
    tslits_dict = tSlits.run(plate_scale=spectrograph.detector[0]['platescale'])

[INFO]    :: Detecting slit edges in the mstrace image
[INFO]    :: Applying bad pixel mask
[INFO]    :: 30 left edges and 26 right edges were found in the trace
[INFO]    :: Assigning slit edge traces
[INFO]    :: Assigning left slit edges
             Outer left edge loop, Iteration 1
               Inner loop, Iteration 1, 11 left edges assigned (11 total)
               Inner loop, Iteration 2, 3 left edges assigned (14 total)
               Inner loop, Iteration 3, 8 left edges assigned (22 total)
               Inner loop, Iteration 4, 2 left edges assigned (24 total)
               Inner loop, Iteration 5, 1 left edges assigned (25 total)
             Outer left edge loop, Iteration 2
               Inner loop, Iteration 1, 11 left edges assigned (11 total)
               Inner loop, Iteration 2, 3 left edges assigned (14 total)
               Inner loop, Iteration 3, 3 left edges assigned (17 total)
               Inner loop, Iteration 4, 3 left edges assigned (20 total)
      

In [10]:
# Look at what TraceSlits was actually trying to trace
viewer, ch = ginga.show_image(tSlits.edgearr)
# Look at the sawtooth convolved image
viewer, ch = ginga.show_image(tSlits.siglev)

tmp = tSlits.edgearr * 100.
tmp[np.where(tmp == 0.)] = 1.
ginga.show_image(tSlits.mstrace * tmp)
ginga.show_slits(viewer, ch, tSlits.lcen, tSlits.rcen, slit_ids = np.arange(tSlits.lcen.shape[1]) + 1, pstep=50)